# Simulación: tiempo discreto (síncrona) vs. tiempo continuo (asíncrona)

---

## 1. Comparar la simulación de tiempo discreto (síncrona) y de tiempo continuo (asíncrona)

### 1.a) Escenarios donde cada una es preferible

#### **Tiempo discreto (síncrona)**
- **Modelos con “ticks” naturales o pasos fijos**: autómatas celulares, modelos de agentes con rondas, simulaciones educativas.
- **Interacciones globales por ronda**: juegos evolutivos, mercados en “batch”, consenso por iteraciones.
- **Fenómenos muestreados**: series temporales a intervalos fijos, control por ciclos ($\Delta t$ fijo).
- **Hardware/GPU friendly**: fácil de vectorizar y paralelizar; todos los agentes actualizan en lockstep.
- **Requisitos de reproducibilidad y depuración**: orden determinista por fases (leer–calcular–escribir) minimiza condiciones de carrera.

#### **Tiempo continuo (asíncrona / de eventos discretos)**
- **Eventos raros pero críticos**: colas/servicios (teoría de colas), redes de comunicaciones, simulaciones de tráfico.
- **Procesos estocásticos continuos**: química (Gillespie), saltos de Markov, fallas y reparaciones con tiempos exponenciales.
- **Cuando la precisión temporal importa**: tiempos exactos de llegada/salida; evitar aliasing de eventos entre ticks.
- **Sistemas escasos (sparse)**: pocos cambios por unidad de tiempo; se evita recorrer todo el estado cada $\Delta t$.
- **Simulaciones con prioridades/causalidad fina**: reglas diferentes para “eventos simultáneos” y resolución de empates.

---

### 1.b) Compensaciones computacionales (velocidad vs. precisión)

| Aspecto | Tiempo discreto (síncrona) | Tiempo continuo (asíncrona) |
|---|---|---|
| **Granularidad temporal** | Paso fijo $\Delta t$ | Tiempos exactos de evento $t_e$ |
| **Costo aproximado** | $\text{Costo} \approx \frac{T}{\Delta t}\cdot C(N)$ (recorre muchos estados en cada tick) | $\text{Costo} \approx E \cdot (\text{coste\_evento})$, típicamente $E \log E$ con colas de prioridad |
| **Precisión** | Error de discretización $\sim O(\Delta t^p)$ (según integrador); aliasing si eventos caen entre ticks | Temporalmente exacta (a resolución de reloj); sin error de paso |
| **Eficiencia en sistemas “escasos”** | Baja (procesa aunque nada cambie) | Alta (solo cuando hay eventos) |
| **Paralelización** | Fácil (lockstep); ideal para GPU/ SIMD | Más compleja (colas compartidas, sincronización) |
| **Complejidad de implementación** | Baja/Media | Media/Alta (gestión de eventos, cancelaciones) |

**Reglas prácticas:**
- Si el fenómeno **cambia lentamente** o puede muestrearse sin perder dinámica: usar **síncrona** con un $\Delta t$ que equilibre precisión y costo.
- Si los **eventos puntuales determinan la dinámica** (llegadas/salidas, reacciones, fallas): usar **asíncrona** de eventos.
- Ajuste de $\Delta t$: disminuir $\Delta t$ **aumenta precisión** pero **incrementa** el costo $\propto 1/\Delta t$. En métodos de orden $p$, el error $\approx O(\Delta t^p)$.
- En **Gillespie/CTMC**: el siguiente tiempo de evento se muestrea como $\tau \sim \mathrm{Exp}(\lambda)$, con $\tau = -\ln(u)/\lambda$ (donde $u\sim \mathcal{U}(0,1)$).

---

## 2. Mecanismos de gestión de eventos

### 2.a) Cómo las colas de eventos gestionan los cambios de estado

**Estructura típica de un evento:** $(t, \text{id}, \text{tipo}, \text{payload})$  
**Cola de prioridad** ordenada por tiempo (y desempates).

**Bucle principal (discreto por eventos):**
1. Extraer el evento con menor tiempo: `e ← pop_min(Q)`.
2. Avanzar el reloj: `t ← e.t`.
3. Aplicar la lógica del evento: actualizar el estado afectado.
4. Programar eventos futuros derivados: `push(Q, e')` con sus tiempos $t' \ge t$.
5. Repetir hasta alcanzar horizonte $T$ o vaciar la cola.

**Detalles importantes:**
- **Estructuras**: heap binario ($\text{push/pop}: O(\log n)$), **calendar queue** o **ladder queue** (esperanza $\approx O(1)$ en distribuciones “bien portadas”), skip list de prioridad.
- **Cancelación/reprogramación**: eventos obsoletos pueden invalidarse (marcas de versión o flags) o retirarse explícitamente.
- **Consistencia**: event handlers deben ser **idempotentes** o protegerse ante duplicados/cancelaciones.
- **Paralelismo (PDES)**: enfoques conservadores u optimistas; en optimista existen **anti-eventos** para revertir (rollback) si se viola causalidad.

---

### 2.b) Gestión de prioridades para eventos concurrentes

Cuando múltiples eventos comparten el **mismo tiempo $t$**, se requiere una política consistente para preservar la causalidad y evitar sesgos:

- **Prioridad compuesta estable**: ordenar por tupla $(t,\ \text{prioridad},\ \text{id})$; asegura determinismo.  
  - Ej.: `prioridad = {CREACIÓN > ACTIVACIÓN > LECTURA > ESCRITURA > MEDICIÓN}`.
- **Tiempo superdenso (DEVS)**: usar pares $(t, k)$ con **micro-pasos** $k=0,1,2,\dots$ para imponer un orden interno sin “mover” el tiempo físico.
- **Fases por barreras**: procesar primero todos los eventos de lectura, luego de actualización, etc., evitando leer estados parcialmente actualizados.
- **Equidad y hambruna**: si ciertas clases de eventos reciben prioridad, introducir **rotación** o **cupos** para evitar starvation.
- **Conmutatividad**: si dos eventos conmutan (aplican a entidades disjuntas), pueden ejecutarse en paralelo; si no, imponer orden total.
- **Determinismo reproducible**: romper empates con una **llave estable** (p.ej., `id`), nunca con aleatorios.
- **Complejidad**: la cola conserva $O(\log n)$ por inserción/extracción (heap). Con calendar queues, la complejidad esperada puede acercarse a $O(1)$; memoria $O(E)$.

